# Part 1 - Step-by-step code walkthrough
The source of this code and explanation is to be found in the following [article](https://blogs.alisterluiz.com/zero-shot-text-classification-in-8-minutes-a-step-by-step-guide/)

- **Importing pipeline:** This high-level function from Hugging Face’s Transformers library simplifies many NLP tasks.
- **Setting up the pipeline:** By specifying `"zero-shot-classification"` as the task and choosing the facebook/bart-large-mnli model, we’re leveraging a model trained for natural language inference (NLI). This model is adept at comparing a given text with candidate labels to determine the best match.

In [1]:
from transformers import pipeline

# Create a zero-shot classifier pipeline using Facebook's BART model fine-tuned on MNLI.
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

config.json: 0.00B [00:00, ?B/s]

c:\Users\Esmee Werk\Documents\Persoonlijke Projecten\AI-Onderzoeksvragen\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Esmee Werk\.cache\huggingface\hub\models--facebook--bart-large-mnli. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' packa

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cpu


- **Input Text:** This is the text review we want to analyze.
- **Candidate Labels:** For this example, we’re classifying the sentiment of the text. The model will score each label based on how well it fits the input.

In [7]:
# Example for Sentiment Analysis
sequence = "I absolutely love the new smartphone design – it’s sleek and super intuitive!"
candidate_labels = ["positive sentiment", "negative sentiment", "neutral"]

- **Running the Classifier:** The classifier function processes the input text along with the candidate labels.
- **Output Structure:** The output is a dictionary that includes:
    - `labels:` The candidate labels ranked from most to least relevant.
    - `scores:` Confidence scores for each label, indicating how likely the text belongs to that category.
    This quick output helps you instantly see which sentiment (or label) best matches the input text.

In [8]:
# Run the classifier and capture the result
result = classifier(sequence, candidate_labels)

# Print the output
print("Sentiment Analysis Result:")
print(result)

Sentiment Analysis Result:
{'sequence': 'I absolutely love the new smartphone design – it’s sleek and super intuitive!', 'labels': ['positive sentiment', 'neutral', 'negative sentiment'], 'scores': [0.9924148321151733, 0.0051809255965054035, 0.0024042511358857155]}


You can also use zero-shot classification for different topics but sentiment analysis. You can adapt the same code for topic categorization or any other task by changing the `candidate_labels`.

Below we use a new sample text related to AI Advancements.
Instead of sentiment, we will provide topic labels. Then the model is going to evaluate which topic best describes the text.

In [9]:
# Example for Topic Classification
sequence_topic = "The recent advancements in artificial intelligence are transforming industries across the board."
candidate_topics = ["technology", "politics", "health", "sports"]

# Run the classifier for topic classification
result_topic = classifier(sequence_topic, candidate_topics)

print("\nTopic Classification Result:")
print(result_topic)


Topic Classification Result:
{'sequence': 'The recent advancements in artificial intelligence are transforming industries across the board.', 'labels': ['technology', 'health', 'sports', 'politics'], 'scores': [0.9800008535385132, 0.01203815545886755, 0.004841970279812813, 0.0031191199086606503]}


## Part 2 - Understanding the technique
In this part of the notebook we take the lessons from the article and take it a step further. In part one we used a black box, now we're trying to understand what the black box does.

The goal of this part is to:

- Explain what `NLI` is (Natural Language Interference)
    * The `facebook/bart-large-mnli` is trained on **entail,ent/contradiction/neutral**
    * Zero-shot becomes a possibility by transforming labels (like "sport") into a 'hypothesis'
        - *Premise*: "The recent advancements in AI are transforming industries."
        - *Hypothesis*: "The text is about sports."
        - The model will decide if Premise or Hypothesis entails, contradicts or is neutral.
- I will show a "handmade" NLI without pipeline
    - For this i will use Hugging Face `AutoTokenizer` and `AutoModelForSequenceClassification` to tokenize my own input en take logits from the model.
    - This shows how you get the probabilities for entailment/contradiction/neutral
    - I will apply this on my own “sports/technology/health” labels.

In [10]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

model_name = "facebook/bart-large-mnli"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

In [16]:
premise = "The recent advancements in AI are transforming industries."
hypotheses = [
    "This text is about technology.",
    "This text is about sports.",
    "This text is about politics."
]

for hypothesis in hypotheses:
    inputs = tokenizer(premise, hypothesis, return_tensors="pt")
    outputs = model(**inputs)
    logits = outputs.logits
    probs = torch.softmax(logits, dim=1).detach().numpy()
    entailment_prob = probs[0][2]  # label 2 is usually entailment
    print(f"Hypothesis: '{hypothesis}'")
    print("Entailment probability:", entailment_prob)
    print("-" * 40)

Hypothesis: 'This text is about technology.'
Entailment probability: 0.56221676
----------------------------------------
Hypothesis: 'This text is about sports.'
Entailment probability: 0.00018156749
----------------------------------------
Hypothesis: 'This text is about politics.'
Entailment probability: 0.00016065556
----------------------------------------


Since the premise and hypothesis concept is still a bit vague heres an in-depth explanation.

A NLI-Model always gets two scentences:

1. **Premise**: This is the given text (what we know) --> "*The recent advancements in AI are transforming industries*"
2. **Hypothesis**: This is the proposition we want to check.

The model will then decide if the **hypothesis** is logical according to the **premies**.

This can give you three results:

* **Entailment**: The hypothesis flows logically from the premise
* **Contradiction**: The hypothesis contradicts the premise
* **Neutral**: The hypothesis can't be confirmed nor denied based on the premise

Below i will show three examples of entailment, contradiction and neutral.

In [20]:
examples = [
    {
        "premise": "A man is playing a guitar on stage.",
        "hypothesis": "A person is performing music.",
        "expected": "Entailment"
    },
    {
        "premise": "A man is playing a guitar on stage.",
        "hypothesis": "A person is cooking dinner.",
        "expected": "Contradiction"
    },
    {
        "premise": "A man is playing a guitar on stage.",
        "hypothesis": "A person is feeling happy.",
        "expected": "Neutral"
    }
]

labels = ["Contradiction", "Neutral", "Entailment"]

for ex in examples:
    inputs = tokenizer(ex["premise"], ex["hypothesis"], return_tensors="pt")
    outputs = model(**inputs)
    logits = outputs.logits
    probs = torch.softmax(logits, dim=1).detach().numpy()[0]
    probs_percent = [round(p * 100, 2) for p in probs]
    predicted = labels[probs.argmax()]
    print(f"Premise: {ex['premise']}")
    print(f"Hypothesis: {ex['hypothesis']}")
    print(f"Predicted: {predicted} (probabilities: {probs_percent}%)")
    print(f"Expected: {ex['expected']}")
    print("-" * 40)

Premise: A man is playing a guitar on stage.
Hypothesis: A person is performing music.
Predicted: Entailment (probabilities: [np.float32(0.01), np.float32(0.37), np.float32(99.62)]%)
Expected: Entailment
----------------------------------------
Premise: A man is playing a guitar on stage.
Hypothesis: A person is cooking dinner.
Predicted: Contradiction (probabilities: [np.float32(99.98), np.float32(0.01), np.float32(0.0)]%)
Expected: Contradiction
----------------------------------------
Premise: A man is playing a guitar on stage.
Hypothesis: A person is feeling happy.
Predicted: Neutral (probabilities: [np.float32(1.05), np.float32(97.56), np.float32(1.39)]%)
Expected: Neutral
----------------------------------------


#### How can this be used for Zero-Shot learning?
With classification you usually have a set list of labels (example:["sports", "politics", "technology"])

With zero-shot you **don't** have a trained model for these specific labels. See the following example:

Premise: "*The recent advancements in artificial intelligence are transforming industries.*"

- **Hypothesis 1:** "This text is about sports."
- **Hypothesis 2:** "This text is about technology."
- **Hypothesis 3:** "This text is about politics."

The NLI-Model will then check per hypothesis: "How probable is entailment?"
    - If the entailment-score is high for "technology", the model will then say:
        * this text is probably about technology

**Why this works**
The NLI-Model is trained to understand **relationships between scentences**. You re-use those skills to do new classification-tasks without extra training. This is why a BART-MNLI can predict labels its never seen before: zero-shot learning.

---

### Summarized

* **Premise:** Your input sentencen
* **Hypothesis**: "This text is about [label]"
* **Entailment**: How probable the model thinks that the premise supports the hypothesis.